In [19]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/07 16:36
# @Author  : Wang Yujia
# @File    : PT_gen.ipynb
# @Description : calculating U&P with inferred params

# 0. What for
1. Demo for,用infer的结果来算U_t

# 1. Preparations
## 1.1 全局设置

## 1.2 data 读取
1. 保存data_key_15
2. 并且提取成功infer的data`params_all`

In [5]:
# small fix from here
# data path
data_selected_path = "../data/SA_PT/datawithnp_PT_selected.csv"
# data key path
data_key_path = "../data/SA_PT/data_key_PT.csv"
# inferred params
params_opitim_delta_wset_path = "../data/SA_PT/params_opitim_delta_T_wset.csv"

# output path
output_path_root = "../data/SA_PT/results/"
filename_head = "PT_all1303_"
filename_tail = ".csv"
filename_P = output_path_root+(filename_head + "P" + filename_tail)

features_GT = ['product_id','bidincrement','bidfee','retail']

# threshold
K = 100

import numpy as np
import csv
import pandas as pd
from visdom import Visdom
from SA_for_PT_funcs_delta_eq1 import *
import json


# viz = Visdom(env='P_1forall',use_incoming_socket=False)

In [2]:
data = pd.read_csv(data_selected_path, encoding="utf-8")
params_opitim_delta_wset = pd.read_csv(params_opitim_delta_wset_path, encoding="utf-8")

idx = (pd.notna(params_opitim_delta_wset.alpha))
params_all = params_opitim_delta_wset[idx][:]
params_all.reset_index(drop=True,inplace=True)
print(f"一共有 *{params_all.shape[0]}* 个auction setting成功infer")
print(f"一共有 *{params_opitim_delta_wset.shape[0] - params_all.shape[0]}* 个auction setting未能成功infer")

data_key = pd.read_csv(data_key_path, encoding="utf-8")

一共有 *1303* 个auction setting成功infer
一共有 *0* 个auction setting未能成功infer


## 1.3 functions about 'key'

In [3]:
# get key from i in 'data_key'
def get_key_from_index(i,flag="NotStr"):
    if(flag == "str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(int(key_i[0])),str(key_i[1]),str(key_i[2]),str(key_i[3]))

        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

#features_GT = ['product_id','bidincrement','bidfee','retail']
def select_data_fromkey(key_i_str):
    return data[(data['product_id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. U&P

1. 得到`params_all`之后，可以对不同的auction settings做generate了
2. generate过程无非是求u-->p，u的代码在上面loss func里写过了。然后把P存到dict里
3. U 由 `f_equi`得到
3. `P[t] = U[0]*U[1]*U[2]...*U[t]*U[t+1]`
4. `P[key_i_str][max_T]==tmp[max_T]*U[max_T]`

In [ ]:
# solve for U&P respectively for all settings
P = {}
# params_all.shape[0]
for i in range(0,params_all.shape[0]):

    # get optimized params
    alpha, labda = params_all.alpha[i],params_all.labda[i]

    # get i_th data_key
    key_i = get_key_from_index(i)
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)

    # Get params
    cnt_n_2_i = data_i['cnt_n_2'].astype(int)       # Number of occurrences of different durations
    v = float(data_i['retail'].unique())            # retail price = valuation
    d = float(data_i['bidincrement'].unique())      # bid increment
    b = float(data_i['bidfee'].unique())            # bid fee
    T_i = int((v-b)/d)                                   # duration limitation

    U = [0] * (T_i + 1)                            # the prob. that someone offers a bid in t_th round
    U[0] = 1
    key_i_str = get_key_from_index(i,"str")         # key for P

    P[key_i_str] = np.array([0.0]*(T_i+1))
    P_tmp = [0.0]*(T_i+1)                         # P is what we want to generate
    P_tmp[0] = 1
    tmp = np.array([0.0]*(T_i+2))                 # tmp的大小其实不需要太精确
    tmp[0] = 1.0

    # solve for U
    for t in range(1,T_i+1):
        U[t] = f_Equi(t, v, d, b, alpha, labda)

    # solve for P
    for t in range(0,T_i):
        tmp[t+1] = tmp[t]*U[t]                      # tmp[t]存了到(t-1)的连乘积
        P[key_i_str][t] = (1-U[t+1])*tmp[t+1]

    P[key_i_str][T_i] = tmp[T_i]*U[T_i]

    # if(i%10 == 0):
    #     viz.line(P[key_i_str],np.arange(0,P[key_i_str].shape[0]),win = 'P_'+str(i),opts= dict(title = f'P_{i}_v={v}_b={b}_d={d}'))
print("Done")

## 2.1 Add threshold


In [14]:
# waited to be filled

1288

## 2.2 save
1. 保存结果以及viz环境

In [ ]:
# 保存整个环境
viz.save(envs=['P'])

In [6]:
with open(filename_P,"w+",encoding='utf—8',newline='') as f:
    w= csv.DictWriter(f,fieldnames=['product_id','bidincrement','bidfee','retail','P'])
    w.writeheader()

    key=list(P.keys())
    value =list(P.values())
    for i in range(len(key)):
        dic = {       #字典类型
            'product_id':key[i][0],
            'bidincrement':key[i][1],
            'bidfee':key[i][2],
            'retail':key[i][3],
            'P': list(value[i])
        }
        w.writerow(dic)   #数据写入csv文件

print("Done")

Done


In [103]:
import json
fileName_viz = "../data/vis/env_P.json"
# 保存指定的win

def saveVisdomData(win, env, fileName, mode='w'):
	'''
		模型训练完或中断时，可以先将前阶段visdom的数据保存到本地
		arg:
			win:　窗口名称
			env:　环境名称
			fileName:　保存文件路径
			mode:　文件保存格式，‘w’表示重写，‘a’表示添加在末端
	'''
    # rewrite or append
    assert mode=='w' or mode=='a'
    viz = Visdom()
    win_data = viz.get_window_data(win, env)
    pre_data = json.loads(win_data)
    x = pre_data["content"]["data"][0]["x"]  # x坐标的值
    y1 = pre_data["content"]["data"][0]["y"]  # 曲线１
    y2 = pre_data["content"]["data"][1]["y"]  # 曲线２
    assert len(x)==len(y1)==len(y2)
    with open(fileName_viz, mode) as f:
        writer = csv.writer(f)
        for i in range(len(x)):
            writer.writerow([x[i], y1[i], y2[i]])

saveVisdomData()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

# 3. 还原viz环境

In [11]:

viz_path = "../data/vis/P.json"
with open(viz_path,"r") as f:
    pre_data=json.load(f)

tmp = pre_data['jsons'].values()
for i in range(0,len((pre_data['jsons']).keys())):
    tmpi = list(tmp)[i]
    viz.line(
        X=np.array(tmpi['content']["data"][0]["x"]),
        Y=np.array(tmpi['content']["data"][0]["y"]),
        env = "P",
        win = tmpi['id'],
        opts= dict(title=tmpi['title']),
    )